In [194]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
from captum.attr import *
import quantus
import gc
import os
import sklearn
from sklearn.metrics import roc_auc_score
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import torch.nn.functional as F


In [195]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [196]:
import warnings
warnings.filterwarnings('ignore')

In [197]:
class Network(nn.Module):
    ''' A basic neural network model '''
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()         #python2 : super(MLP, self).__init__()
        #defining the network's operations
        self.fc1 = nn.Linear(input_size, hidden_size[0])
        self.fc2 = nn.Linear(hidden_size[0], hidden_size[1])
        self.fc3 = nn.Linear(hidden_size[1], output_size)

    def forward(self, x, softmax=False): 
        x = x.reshape(-1,83)
        a = self.fc3(F.relu(self.fc2(F.relu(self.fc1(x.float())))))
        if softmax:
            y_pred = F.softmax(a, dim=1)
        else:
            y_pred = a
        #print(y_pred.shape)
        return y_pred

In [199]:
input_size = 83
hidden_size=[64,64]
output_size=2
model = Network(input_size, hidden_size, output_size)
model.load_state_dict(torch.load("./model_bce_loss.pytorch"))
model.to(device)

Network(
  (fc1): Linear(in_features=83, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=2, bias=True)
)

In [200]:
df = pd.read_csv('clean_data_before_scaling_class_0.csv')
df

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,74,75,76,77,78,79,80,81,82,83
0,0,17.0,23999.0,1.0,1.0,77.0,159.0,77.0,77.0,77.000000,...,0.0,0.000000e+00,0.000000,0.0,0.0,-1.0,-1.0,0.0,-1.0,0.0
1,1,6.0,115405386.0,32.0,34.0,703.0,34044.0,337.0,0.0,21.968750,...,39203.0,9.973434e+06,57446.079336,9997094.0,9800577.0,-1.0,-1.0,115405386.0,-1.0,0.0
2,2,17.0,59062.0,2.0,2.0,70.0,216.0,35.0,35.0,35.000000,...,0.0,0.000000e+00,0.000000,0.0,0.0,-1.0,-1.0,0.0,-1.0,0.0
3,3,6.0,14766461.0,23.0,30.0,893.0,47056.0,517.0,0.0,38.826087,...,4619105.0,1.000091e+07,0.000000,10000912.0,10000912.0,-1.0,-1.0,14766461.0,-1.0,0.0
4,4,6.0,5907768.0,4.0,2.0,0.0,0.0,0.0,0.0,0.000000,...,34442.0,5.838974e+06,0.000000,5838974.0,5838974.0,-1.0,-1.0,5907768.0,-1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395596,395596,17.0,32231.0,2.0,2.0,70.0,192.0,35.0,35.0,35.000000,...,0.0,0.000000e+00,0.000000,0.0,0.0,-1.0,-1.0,0.0,-1.0,0.0
395597,395597,6.0,115643087.0,22.0,20.0,995.0,4380.0,613.0,0.0,45.227273,...,23475.0,1.001795e+07,19260.895758,10024497.0,9960011.0,-1.0,-1.0,115643087.0,-1.0,0.0
395598,395598,17.0,122275.0,1.0,1.0,50.0,136.0,50.0,50.0,50.000000,...,0.0,0.000000e+00,0.000000,0.0,0.0,-1.0,-1.0,0.0,-1.0,0.0
395599,395599,6.0,115442585.0,23.0,20.0,745.0,7368.0,391.0,0.0,32.391304,...,23238.0,1.019040e+07,66002.373598,10216763.0,10003600.0,-1.0,-1.0,115442585.0,-1.0,0.0


- True positive is when adversarial is identified as malware. 
- False positive is when benign is identified as malware. 

In [219]:
def compute_TPR(adv, th): 
    TN=0
    FN=0
    FP = 0 
    TP=0
    
    for value in adv: 
        if value<th:
            TP += 1
        else: 
            FN += 1
    return (TP/(TP+FN))*100

In [220]:
def compute_FPR(ben, th): 
    TN=0
    FN=0
    FP = 0 
    TP=0
    
    for value in ben: 
        if value>th:
            TN += 1
        else: 
            FP += 1
    return (FP/(FP+TN))*100


In [203]:
def compute_iqr(attr):
    scores = []
    for i in range(len(attr)):
        a = attr[i].flatten()
        score_75 = np.percentile(a, 75)
        score_25 = np.percentile(a, 25)
        score_qt = score_75 - score_25
        scores.append(score_qt)
    return scores

In [204]:
def train_fs(model, x_batch, y_batch, train_fpr):
    distances = get_attr(model, x_batch, y_batch)
    selected_distance_idx = int(np.ceil(len(x_batch) * (1-train_fpr)))
    threshold = sorted(distances)[selected_distance_idx-1]
    threshold = threshold
    return threshold

def get_attr(model, x_batch, y_batch):
    ig = IntegratedGradients(model)
    a_batch_intgrad = ig.attribute(inputs=x_batch, target=y_batch).to('cpu') 
    iqr = compute_iqr(a_batch_intgrad)
    return iqr

In [205]:
#evaluate on benign samples and save metrics
def compute_benign_metrics(test_loader, model):
    fs = []
    i=0
    for image, label in test_loader:
        i += len(image)
        image, label = image.to(device), label.to(device)
        d = get_attr(model, image, label)    
        fs.extend(d)
        if i>1000:
            return fs 
    return fs


#evaluate on adv samples and save metrics
def compute_adv_metrics(test_loader, model):
    fs = []
    i=0
    for image, label in test_loader: 
        i += len(image) 
        image, label = image.to(device), label.to(device)
        d = get_attr(model, image, label)    
        fs.extend(d)
        if i>1000:
            return fs 
    return fs

In [206]:
class CustomDataset(Dataset):
    def __init__(self, csv_file):
        self.data = pd.read_csv(csv_file)
        self.data = self.data.drop(self.data.columns[0], axis=1)  # Remove unnecessary index column
        self.features = self.data.iloc[:, :-1].values  # Features (all columns except the last)
        self.labels = self.data.iloc[:, -1].values     # Labels (last column)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return torch.tensor(self.features[idx], dtype=torch.float32), torch.tensor(self.labels[idx], dtype=torch.long)

# Load CSV file and split into train and test sets
dataset = CustomDataset('clean_data_before_scaling_class_0.csv')
train_data, test_data = train_test_split(dataset, test_size=0.2, random_state=42)

# Create data loaders for training and testing
batch_size = 64
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

In [221]:
FPR = [0.01,0.05,0.1]
final_th = []
for fpr in FPR:
    t=[]
    for step, (x_batch, y_batch) in enumerate(train_loader):
        
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)
        #print(x_batch.shape)
        #print(y_batch)
        threshold = train_fs(model, x_batch, y_batch, fpr)
        t.append(threshold.item())
        if step==50:
            break
    th = sum(t)/len(t)
    print("Threshold for {} FPR is {}.".format(fpr, th))
    final_th.append(th)

Threshold for 0.01 FPR is 1039.0377512001608.
Threshold for 0.05 FPR is 321.25987808878375.
Threshold for 0.1 FPR is 195.2231011156793.


In [224]:
threshold = [1000,800,600,400,200,100,50,20,10,5,2,1,0.5,0.005,0.0005]

In [225]:
sc = []

dist_ben = compute_benign_metrics(test_loader, model)

advdataset = CustomDataset('adversarial_examples_FGSM1.csv')
advtrain_data, advtest_data = train_test_split(advdataset, test_size=0.2, random_state=42)

# Create data loaders for training and testing
batch_size = 64
train_loader_adv = DataLoader(advtrain_data, batch_size=batch_size, shuffle=True)
test_loader_adv = DataLoader(advtest_data, batch_size=batch_size, shuffle=False)

dist_adv = compute_adv_metrics(test_loader_adv, model)

fpr_results = []
for th in threshold:
    FPR = compute_FPR(dist_ben, th)
    fpr_results.append(FPR/100)
tpr_results = []
for th in threshold:
    TPR = compute_TPR(dist_adv, th)
    tpr_results.append(TPR/100)

print(fpr_results)
print(tpr_results)
score = sklearn.metrics.auc(fpr_results, tpr_results)
print(score)
sc.append(score)

[0.990234375, 0.9873046875, 0.9794921875, 0.9609375, 0.8955078125, 0.8046875, 0.7001953125, 0.6240234375, 0.5732421875, 0.49609375, 0.361328125, 0.21484375, 0.203125, 0.1904296875, 0.1904296875]
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.98, 0.805, 0.435, 0.0, 0.0]
0.76832275390625


In [226]:
dist_ben = compute_benign_metrics(test_loader, model)

advdataset = CustomDataset('adversarial_examples_FGSM2.csv')
advtrain_data, advtest_data = train_test_split(advdataset, test_size=0.2, random_state=42)

# Create data loaders for training and testing
batch_size = 64
train_loader_adv = DataLoader(advtrain_data, batch_size=batch_size, shuffle=True)
test_loader_adv = DataLoader(advtest_data, batch_size=batch_size, shuffle=False)

dist_adv = compute_adv_metrics(test_loader_adv, model)

fpr_results = []
for th in threshold:
    FPR = compute_FPR(dist_ben, th)
    fpr_results.append(FPR/100)
tpr_results = []
for th in threshold:
    TPR = compute_TPR(dist_adv, th)
    tpr_results.append(TPR/100)

score = sklearn.metrics.auc(fpr_results, tpr_results)
print(score)
sc.append(score)

dist_ben = compute_benign_metrics(test_loader, model)

advdataset = CustomDataset('adversarial_examples_FGSM3.csv')
advtrain_data, advtest_data = train_test_split(advdataset, test_size=0.2, random_state=42)

# Create data loaders for training and testing
batch_size = 64
train_loader_adv = DataLoader(advtrain_data, batch_size=batch_size, shuffle=True)
test_loader_adv = DataLoader(advtest_data, batch_size=batch_size, shuffle=False)

dist_adv = compute_adv_metrics(test_loader_adv, model)

fpr_results = []
for th in threshold:
    FPR = compute_FPR(dist_ben, th)
    fpr_results.append(FPR/100)
tpr_results = []
for th in threshold:
    TPR = compute_TPR(dist_adv, th)
    tpr_results.append(TPR/100)

score = sklearn.metrics.auc(fpr_results, tpr_results)
print(score)
sc.append(score)

print('---')
dist_ben = compute_benign_metrics(test_loader, model)

advdataset = CustomDataset('adversarial_examples_FGSM4.csv')
advtrain_data, advtest_data = train_test_split(advdataset, test_size=0.2, random_state=42)
batch_size = 64
train_loader_adv = DataLoader(advtrain_data, batch_size=batch_size, shuffle=True)
test_loader_adv = DataLoader(advtest_data, batch_size=batch_size, shuffle=False)

dist_adv = compute_adv_metrics(test_loader_adv, model)

fpr_results = []
for th in threshold:
    FPR = compute_FPR(dist_ben, th)
    fpr_results.append(FPR/100)
tpr_results = []
for th in threshold:
    TPR = compute_TPR(dist_adv, th)
    tpr_results.append(TPR/100)

score = sklearn.metrics.auc(fpr_results, tpr_results)
print(score)
sc.append(score)

print('---')
dist_ben = compute_benign_metrics(test_loader, model)

advdataset = CustomDataset('adversarial_examples_FGSM5.csv')
advtrain_data, advtest_data = train_test_split(advdataset, test_size=0.2, random_state=42)
batch_size = 64
train_loader_adv = DataLoader(advtrain_data, batch_size=batch_size, shuffle=True)
test_loader_adv = DataLoader(advtest_data, batch_size=batch_size, shuffle=False)

dist_adv = compute_adv_metrics(test_loader_adv, model)

fpr_results = []
for th in threshold:
    FPR = compute_FPR(dist_ben, th)
    fpr_results.append(FPR/100)
tpr_results = []
for th in threshold:
    TPR = compute_TPR(dist_adv, th)
    tpr_results.append(TPR/100)

score = sklearn.metrics.auc(fpr_results, tpr_results)
print(score)
sc.append(score)

0.74116455078125
0.6940429687499999
---
0.6641259765624999
---
0.61629638671875


In [227]:
print(np.mean(sc),np.std(sc))

0.69679052734375 0.05411052027830608


In [228]:
sc = []

dist_ben = compute_benign_metrics(test_loader, model)

advdataset = CustomDataset('adversarial_examples_PGD1.csv')
advtrain_data, advtest_data = train_test_split(advdataset, test_size=0.2, random_state=42)

# Create data loaders for training and testing
batch_size = 64
train_loader_adv = DataLoader(advtrain_data, batch_size=batch_size, shuffle=True)
test_loader_adv = DataLoader(advtest_data, batch_size=batch_size, shuffle=False)

dist_adv = compute_adv_metrics(test_loader_adv, model)

fpr_results = []
for th in threshold:
    FPR = compute_FPR(dist_ben, th)
    fpr_results.append(FPR/100)
tpr_results = []
for th in threshold:
    TPR = compute_TPR(dist_adv, th)
    tpr_results.append(TPR/100)

score = sklearn.metrics.auc(fpr_results, tpr_results)
print(score)
sc.append(score)

print('---')
dist_ben = compute_benign_metrics(test_loader, model)
advdataset = CustomDataset('adversarial_examples_PGD2.csv')
advtrain_data, advtest_data = train_test_split(advdataset, test_size=0.2, random_state=42)

# Create data loaders for training and testing
batch_size = 64
train_loader_adv = DataLoader(advtrain_data, batch_size=batch_size, shuffle=True)
test_loader_adv = DataLoader(advtest_data, batch_size=batch_size, shuffle=False)

dist_adv = compute_adv_metrics(test_loader_adv, model)

fpr_results = []
for th in threshold:
    FPR = compute_FPR(dist_ben, th)
    fpr_results.append(FPR/100)
tpr_results = []
for th in threshold:
    TPR = compute_TPR(dist_adv, th)
    tpr_results.append(TPR/100)

score = sklearn.metrics.auc(fpr_results, tpr_results)
print(score)
sc.append(score)

print('---')
dist_ben = compute_benign_metrics(test_loader, model)
advdataset = CustomDataset('adversarial_examples_PGD3.csv')
advtrain_data, advtest_data = train_test_split(advdataset, test_size=0.2, random_state=42)

# Create data loaders for training and testing
batch_size = 64
train_loader_adv = DataLoader(advtrain_data, batch_size=batch_size, shuffle=True)
test_loader_adv = DataLoader(advtest_data, batch_size=batch_size, shuffle=False)

dist_adv = compute_adv_metrics(test_loader_adv, model)

fpr_results = []
for th in threshold:
    FPR = compute_FPR(dist_ben, th)
    fpr_results.append(FPR/100)
tpr_results = []
for th in threshold:
    TPR = compute_TPR(dist_adv, th)
    tpr_results.append(TPR/100)

score = sklearn.metrics.auc(fpr_results, tpr_results)
print(score)
sc.append(score)

print('---')
dist_ben = compute_benign_metrics(test_loader, model)
advdataset = CustomDataset('adversarial_examples_PGD4.csv')
advtrain_data, advtest_data = train_test_split(advdataset, test_size=0.2, random_state=42)

# Create data loaders for training and testing
batch_size = 64
train_loader_adv = DataLoader(advtrain_data, batch_size=batch_size, shuffle=True)
test_loader_adv = DataLoader(advtest_data, batch_size=batch_size, shuffle=False)

dist_adv = compute_adv_metrics(test_loader_adv, model)

fpr_results = []
for th in threshold:
    FPR = compute_FPR(dist_ben, th)
    fpr_results.append(FPR/100)
tpr_results = []
for th in threshold:
    TPR = compute_TPR(dist_adv, th)
    tpr_results.append(TPR/100)

score = sklearn.metrics.auc(fpr_results, tpr_results)
print(score)
sc.append(score)

print('---')
dist_ben = compute_benign_metrics(test_loader, model)
advdataset = CustomDataset('adversarial_examples_PGD5.csv')
advtrain_data, advtest_data = train_test_split(advdataset, test_size=0.2, random_state=42)

# Create data loaders for training and testing
batch_size = 64
train_loader_adv = DataLoader(advtrain_data, batch_size=batch_size, shuffle=True)
test_loader_adv = DataLoader(advtest_data, batch_size=batch_size, shuffle=False)

dist_adv = compute_adv_metrics(test_loader_adv, model)

fpr_results = []
for th in threshold:
    FPR = compute_FPR(dist_ben, th)
    fpr_results.append(FPR/100)
tpr_results = []
for th in threshold:
    TPR = compute_TPR(dist_adv, th)
    tpr_results.append(TPR/100)

score = sklearn.metrics.auc(fpr_results, tpr_results)
print(score)
sc.append(score)

0.75349609375
---
0.69554443359375
---
0.6350341796875001
---
0.6022778320312501
---
0.59085205078125


In [229]:
print(np.mean(sc),np.std(sc))

0.65544091796875 0.0610533863126664


In [230]:
sc = []

dist_ben = compute_benign_metrics(test_loader, model)

advdataset = CustomDataset('adversarial_examples_BIM1.csv')
advtrain_data, advtest_data = train_test_split(advdataset, test_size=0.2, random_state=42)

# Create data loaders for training and testing
batch_size = 64
train_loader_adv = DataLoader(advtrain_data, batch_size=batch_size, shuffle=True)
test_loader_adv = DataLoader(advtest_data, batch_size=batch_size, shuffle=False)

dist_adv = compute_adv_metrics(test_loader_adv, model)

fpr_results = []
for th in threshold:
    FPR = compute_FPR(dist_ben, th)
    fpr_results.append(FPR/100)
tpr_results = []
for th in threshold:
    TPR = compute_TPR(dist_adv, th)
    tpr_results.append(TPR/100)


score = sklearn.metrics.auc(fpr_results, tpr_results)
print(score)
sc.append(score)

print('----')
dist_ben = compute_benign_metrics(test_loader, model)

advdataset = CustomDataset('adversarial_examples_BIM2.csv')
advtrain_data, advtest_data = train_test_split(advdataset, test_size=0.2, random_state=42)

# Create data loaders for training and testing
batch_size = 64
train_loader_adv = DataLoader(advtrain_data, batch_size=batch_size, shuffle=True)
test_loader_adv = DataLoader(advtest_data, batch_size=batch_size, shuffle=False)

dist_adv = compute_adv_metrics(test_loader_adv, model)

fpr_results = []
for th in threshold:
    FPR = compute_FPR(dist_ben, th)
    fpr_results.append(FPR/100)
tpr_results = []
for th in threshold:
    TPR = compute_TPR(dist_adv, th)
    tpr_results.append(TPR/100)

score = sklearn.metrics.auc(fpr_results, tpr_results)
print(score)
sc.append(score)

print('----')
dist_ben = compute_benign_metrics(test_loader, model)

advdataset = CustomDataset('adversarial_examples_BIM3.csv')
advtrain_data, advtest_data = train_test_split(advdataset, test_size=0.2, random_state=42)

# Create data loaders for training and testing
batch_size = 64
train_loader_adv = DataLoader(advtrain_data, batch_size=batch_size, shuffle=True)
test_loader_adv = DataLoader(advtest_data, batch_size=batch_size, shuffle=False)

dist_adv = compute_adv_metrics(test_loader_adv, model)

fpr_results = []
for th in threshold:
    FPR = compute_FPR(dist_ben, th)
    fpr_results.append(FPR/100)
tpr_results = []
for th in threshold:
    TPR = compute_TPR(dist_adv, th)
    tpr_results.append(TPR/100)

score = sklearn.metrics.auc(fpr_results, tpr_results)
print(score)
sc.append(score)

print('----')
dist_ben = compute_benign_metrics(test_loader, model)

advdataset = CustomDataset('adversarial_examples_BIM4.csv')
advtrain_data, advtest_data = train_test_split(advdataset, test_size=0.2, random_state=42)

# Create data loaders for training and testing
batch_size = 64
train_loader_adv = DataLoader(advtrain_data, batch_size=batch_size, shuffle=True)
test_loader_adv = DataLoader(advtest_data, batch_size=batch_size, shuffle=False)

dist_adv = compute_adv_metrics(test_loader_adv, model)

fpr_results = []
for th in threshold:
    FPR = compute_FPR(dist_ben, th)
    fpr_results.append(FPR/100)
tpr_results = []
for th in threshold:
    TPR = compute_TPR(dist_adv, th)
    tpr_results.append(TPR/100)

score = sklearn.metrics.auc(fpr_results, tpr_results)
print(score)
sc.append(score)

print('----')
dist_ben = compute_benign_metrics(test_loader, model)

advdataset = CustomDataset('adversarial_examples_BIM5.csv')
advtrain_data, advtest_data = train_test_split(advdataset, test_size=0.2, random_state=42)

# Create data loaders for training and testing
batch_size = 64
train_loader_adv = DataLoader(advtrain_data, batch_size=batch_size, shuffle=True)
test_loader_adv = DataLoader(advtest_data, batch_size=batch_size, shuffle=False)

dist_adv = compute_adv_metrics(test_loader_adv, model)

fpr_results = []
for th in threshold:
    FPR = compute_FPR(dist_ben, th)
    fpr_results.append(FPR/100)
tpr_results = []
for th in threshold:
    TPR = compute_TPR(dist_adv, th)
    tpr_results.append(TPR/100)

score = sklearn.metrics.auc(fpr_results, tpr_results)
print(score)
sc.append(score)

0.7648193359375
----
0.7648193359375
----
0.7648193359375
----
0.7648193359375
----
0.7648193359375


In [231]:
print(np.mean(sc),np.std(sc))

0.7648193359375 0.0


In [232]:
sc = []

dist_ben = compute_benign_metrics(test_loader, model)

advdataset = CustomDataset('adversarial_examples_CW1.csv')
advtrain_data, advtest_data = train_test_split(advdataset, test_size=0.2, random_state=42)

# Create data loaders for training and testing
batch_size = 64
train_loader_adv = DataLoader(advtrain_data, batch_size=batch_size, shuffle=True)
test_loader_adv = DataLoader(advtest_data, batch_size=batch_size, shuffle=False)

dist_adv = compute_adv_metrics(test_loader_adv, model)

fpr_results = []
for th in threshold:
    FPR = compute_FPR(dist_ben, th)
    fpr_results.append(FPR/100)
tpr_results = []
for th in threshold:
    TPR = compute_TPR(dist_adv, th)
    tpr_results.append(TPR/100)

score = sklearn.metrics.auc(fpr_results, tpr_results)
print(score)
sc.append(score)

print('----')

dist_ben = compute_benign_metrics(test_loader, model)

advdataset = CustomDataset('adversarial_examples_CW2.csv')
advtrain_data, advtest_data = train_test_split(advdataset, test_size=0.2, random_state=42)

# Create data loaders for training and testing
batch_size = 64
train_loader_adv = DataLoader(advtrain_data, batch_size=batch_size, shuffle=True)
test_loader_adv = DataLoader(advtest_data, batch_size=batch_size, shuffle=False)

dist_adv = compute_adv_metrics(test_loader_adv, model)

fpr_results = []
for th in threshold:
    FPR = compute_FPR(dist_ben, th)
    fpr_results.append(FPR/100)
tpr_results = []
for th in threshold:
    TPR = compute_TPR(dist_adv, th)
    tpr_results.append(TPR/100)

score = sklearn.metrics.auc(fpr_results, tpr_results)
print(score)
sc.append(score)

print('----')

dist_ben = compute_benign_metrics(test_loader, model)

advdataset = CustomDataset('adversarial_examples_CW3.csv')
advtrain_data, advtest_data = train_test_split(advdataset, test_size=0.2, random_state=42)

# Create data loaders for training and testing
batch_size = 64
train_loader_adv = DataLoader(advtrain_data, batch_size=batch_size, shuffle=True)
test_loader_adv = DataLoader(advtest_data, batch_size=batch_size, shuffle=False)

dist_adv = compute_adv_metrics(test_loader_adv, model)

fpr_results = []
for th in threshold:
    FPR = compute_FPR(dist_ben, th)
    fpr_results.append(FPR/100)
tpr_results = []
for th in threshold:
    TPR = compute_TPR(dist_adv, th)
    tpr_results.append(TPR/100)

score = sklearn.metrics.auc(fpr_results, tpr_results)
print(score)
sc.append(score)

print('----')

dist_ben = compute_benign_metrics(test_loader, model)

advdataset = CustomDataset('adversarial_examples_CW4.csv')
advtrain_data, advtest_data = train_test_split(advdataset, test_size=0.2, random_state=42)

# Create data loaders for training and testing
batch_size = 64
train_loader_adv = DataLoader(advtrain_data, batch_size=batch_size, shuffle=True)
test_loader_adv = DataLoader(advtest_data, batch_size=batch_size, shuffle=False)

dist_adv = compute_adv_metrics(test_loader_adv, model)

fpr_results = []
for th in threshold:
    FPR = compute_FPR(dist_ben, th)
    fpr_results.append(FPR/100)
tpr_results = []
for th in threshold:
    TPR = compute_TPR(dist_adv, th)
    tpr_results.append(TPR/100)

score = sklearn.metrics.auc(fpr_results, tpr_results)
print(score)
sc.append(score)

print('----')

dist_ben = compute_benign_metrics(test_loader, model)

advdataset = CustomDataset('adversarial_examples_CW5.csv')
advtrain_data, advtest_data = train_test_split(advdataset, test_size=0.2, random_state=42)

# Create data loaders for training and testing
batch_size = 64
train_loader_adv = DataLoader(advtrain_data, batch_size=batch_size, shuffle=True)
test_loader_adv = DataLoader(advtest_data, batch_size=batch_size, shuffle=False)

dist_adv = compute_adv_metrics(test_loader_adv, model)

fpr_results = []
for th in threshold:
    FPR = compute_FPR(dist_ben, th)
    fpr_results.append(FPR/100)
tpr_results = []
for th in threshold:
    TPR = compute_TPR(dist_adv, th)
    tpr_results.append(TPR/100)

score = sklearn.metrics.auc(fpr_results, tpr_results)
print(score)
sc.append(score)

0.75978515625
----
0.7584716796875
----
0.75853271484375
----
0.75749755859375
----
0.75670654296875


In [233]:
print(np.mean(sc),np.std(sc))

0.7581987304687499 0.0010412549285955229


In [234]:
sc = []

dist_ben = compute_benign_metrics(test_loader, model)
advdataset = CustomDataset('adversarial_examples_autoPGD.csv')
advtrain_data, advtest_data = train_test_split(advdataset, test_size=0.2, random_state=42)
batch_size = 64
train_loader_adv = DataLoader(advtrain_data, batch_size=batch_size, shuffle=True)
test_loader_adv = DataLoader(advtest_data, batch_size=batch_size, shuffle=False)
dist_adv = compute_adv_metrics(test_loader_adv, model)
fpr_results = []
for th in threshold:
    FPR = compute_FPR(dist_ben, th)
    fpr_results.append(FPR/100)
tpr_results = []
for th in threshold:
    TPR = compute_TPR(dist_adv, th)
    tpr_results.append(TPR/100)


score = sklearn.metrics.auc(fpr_results, tpr_results)
print(score)
sc.append(score)

print('----')

dist_ben = compute_benign_metrics(test_loader, model)
advdataset = CustomDataset('adversarial_examples_autoPGD2.csv')
advtrain_data, advtest_data = train_test_split(advdataset, test_size=0.2, random_state=42)
batch_size = 64
train_loader_adv = DataLoader(advtrain_data, batch_size=batch_size, shuffle=True)
test_loader_adv = DataLoader(advtest_data, batch_size=batch_size, shuffle=False)
dist_adv = compute_adv_metrics(test_loader_adv, model)
fpr_results = []
for th in threshold:
    FPR = compute_FPR(dist_ben, th)
    fpr_results.append(FPR/100)
tpr_results = []
for th in threshold:
    TPR = compute_TPR(dist_adv, th)
    tpr_results.append(TPR/100)


score = sklearn.metrics.auc(fpr_results, tpr_results)
print(score)
sc.append(score)

print('----')

dist_ben = compute_benign_metrics(test_loader, model)
advdataset = CustomDataset('adversarial_examples_autoPGD3.csv')
advtrain_data, advtest_data = train_test_split(advdataset, test_size=0.2, random_state=42)
batch_size = 64
train_loader_adv = DataLoader(advtrain_data, batch_size=batch_size, shuffle=True)
test_loader_adv = DataLoader(advtest_data, batch_size=batch_size, shuffle=False)
dist_adv = compute_adv_metrics(test_loader_adv, model)
fpr_results = []
for th in threshold:
    FPR = compute_FPR(dist_ben, th)
    fpr_results.append(FPR/100)
tpr_results = []
for th in threshold:
    TPR = compute_TPR(dist_adv, th)
    tpr_results.append(TPR/100)


score = sklearn.metrics.auc(fpr_results, tpr_results)
print(score)
sc.append(score)

print('----')

dist_ben = compute_benign_metrics(test_loader, model)
advdataset = CustomDataset('adversarial_examples_autoPGD4.csv')
advtrain_data, advtest_data = train_test_split(advdataset, test_size=0.2, random_state=42)
batch_size = 64
train_loader_adv = DataLoader(advtrain_data, batch_size=batch_size, shuffle=True)
test_loader_adv = DataLoader(advtest_data, batch_size=batch_size, shuffle=False)
dist_adv = compute_adv_metrics(test_loader_adv, model)
fpr_results = []
for th in threshold:
    FPR = compute_FPR(dist_ben, th)
    fpr_results.append(FPR/100)
tpr_results = []
for th in threshold:
    TPR = compute_TPR(dist_adv, th)
    tpr_results.append(TPR/100)


score = sklearn.metrics.auc(fpr_results, tpr_results)
print(score)
sc.append(score)
print('----')

dist_ben = compute_benign_metrics(test_loader, model)
advdataset = CustomDataset('adversarial_examples_autoPGD5.csv')
advtrain_data, advtest_data = train_test_split(advdataset, test_size=0.2, random_state=42)
batch_size = 64
train_loader_adv = DataLoader(advtrain_data, batch_size=batch_size, shuffle=True)
test_loader_adv = DataLoader(advtest_data, batch_size=batch_size, shuffle=False)
dist_adv = compute_adv_metrics(test_loader_adv, model)
fpr_results = []
for th in threshold:
    FPR = compute_FPR(dist_ben, th)
    fpr_results.append(FPR/100)
tpr_results = []
for th in threshold:
    TPR = compute_TPR(dist_adv, th)
    tpr_results.append(TPR/100)


score = sklearn.metrics.auc(fpr_results, tpr_results)
print(score)
sc.append(score)


0.62978759765625
----
0.6295410156250001
----
0.62996337890625
----
0.6297607421875001
----
0.62917236328125


In [235]:
print(np.mean(sc),np.std(sc))

0.6296450195312501 0.00027176964132432247
